In [1]:
from transformers import AutoProcessor, AutoModelForImageTextToText

processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM2-2.2B-Instruct")
model = AutoModelForImageTextToText.from_pretrained("HuggingFaceTB/SmolVLM2-2.2B-Instruct")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
model.to("mps")

SmolVLMForConditionalGeneration(
  (model): SmolVLMModel(
    (vision_model): SmolVLMVisionTransformer(
      (embeddings): SmolVLMVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(729, 1152)
      )
      (encoder): SmolVLMEncoder(
        (layers): ModuleList(
          (0-26): 27 x SmolVLMEncoderLayer(
            (self_attn): SmolVLMVisionAttention(
              (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): SmolVLMVisionMLP(
              (activation_fn): PytorchGELUTanh()
              (fc1): Linear(in_features

In [10]:
from PIL import Image
import io

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": "../files/image.png"},
            {"type": "text", "text": "Answer using digit only. How many are circles in the image?"}
        ]
    },
]

In [14]:
import torch

model.config.attn_implementation = "eager"

inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

outputs = model.generate(
    **inputs, 
    do_sample=False, 
    max_new_tokens=64, 
    output_attentions=True, 
    return_dict_in_generate=True
)
generated_ids = outputs.sequences
attentions = outputs.attentions
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

In [16]:
print(generated_texts[0])

User:




Answer using digit only. How many are circles in the image?
Assistant: 2


In [18]:
print(attentions)

(None, None, None)
